In [ ]:
pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.

In [ ]:
import pandas as pd
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from umap import UMAP
from sentence_transformers import SentenceTransformer
import nltk
from nltk.corpus import stopwords

In [ ]:
df = pd.read_csv("/content/cases.csv")
df.head()

,id,type,title,url,description,featured,post_date,updated_date,creator_id,creator_name,...,formal_evaluation,body,files_count,links_count,photos_count,videos_count,audio_count,evaluation_reports_count,evaluation_links_count,collections
0,1,case,British Columbia Citizens' Assembly on Elector...,https://participedia.net/case/1,"An independent, non-partisan assembly of 160 r...",0,2009-09-16,2021-04-12,4,Arfung,...,1.0,Problems and Purpose The Citizens' Assembly...,762,53,312,216,2,2,2,"(6786,collection,""OECD Project on Representati..."
1,2,case,Minneapolis Neighborhood Revitalization Program,https://participedia.net/case/2,The Minneapolis Neighborhood Revitalization Pr...,0,2009-09-17,2020-08-17,14,Annayork,...,NaN,Problems and Purpose The Minneapolis Neighbo...,2,1220,191,2,2,2,2,NaN
2,3,case,Wenling City Deliberative Poll,https://participedia.net/case/3,One of China’s most innovative forms of reinve...,0,2009-10-23,2020-08-04,24,Swdgray,...,1.0,"Problems and Purpose In late 2004, local gov...",2,217,235,2,2,2,274,"(6786,collection,""OECD Project on Representati..."
3,4,case,Citizens' Forum Europe,https://participedia.net/case/4,Citizens' Forum Europe was designed to allow d...,0,2009-11-04,2020-08-27,1027,"""Patrizia Nanz""",...,NaN,Problems and Purpose Fuelled by EU skepticis...,2,305,121,2,2,2,2,NaN
4,5,case,Biobanking in British Columbia,https://participedia.net/case/5,Two deliberative forums involving members of t...,0,2009-11-26,2019-11-27,32,Kcodoher,...,1.0,Problems and Purpose Biobanks have been hail...,2,2,120,2,2,2,497,NaN


In [ ]:
columns = ["title", "description", "body"]
df = df[columns]
df.head()

,title,description,body
0,British Columbia Citizens' Assembly on Elector...,"An independent, non-partisan assembly of 160 r...",Problems and Purpose The Citizens' Assembly...
1,Minneapolis Neighborhood Revitalization Program,The Minneapolis Neighborhood Revitalization Pr...,Problems and Purpose The Minneapolis Neighbo...
2,Wenling City Deliberative Poll,One of China’s most innovative forms of reinve...,"Problems and Purpose In late 2004, local gov..."
3,Citizens' Forum Europe,Citizens' Forum Europe was designed to allow d...,Problems and Purpose Fuelled by EU skepticis...
4,Biobanking in British Columbia,Two deliberative forums involving members of t...,Problems and Purpose Biobanks have been hail...


In [ ]:
# Combine text fields for better topic modeling
df["text"] = df["description"].fillna("") + " " + df["body"].fillna("")+" "+df["title"].fillna("")

In [ ]:
# Initialize a sentence transformer model for embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Convert text to embeddings
embeddings = embedding_model.encode(df["text"].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/71 [00:00<?, ?it/s]

In [ ]:
# Initialize UMAP for dimensionality reduction
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.1, metric='cosine')

In [ ]:
# Download NLTK stop words
nltk.download("stopwords")
custom_stop_words = stopwords.words("english")

# Initialize vectorizer with stop words
vectorizer_model = CountVectorizer(stop_words=custom_stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Create BERTopic model
topic_model = BERTopic(embedding_model=embedding_model, umap_model=umap_model, vectorizer_model=vectorizer_model)

In [ ]:
# Fit model to text data
topics, probs = topic_model.fit_transform(df["text"].tolist(), embeddings)

In [ ]:
# Add extracted topic info to the dataframe
df["topic"] = topics
df["topic_name"] = df["topic"].apply(lambda x: topic_model.get_topic(x) if x != -1 else "No Topic")

In [ ]:
# Extract keywords for each topic
topic_keywords = topic_model.get_topics()

In [ ]:
# Display top topics
print(topic_model.get_topic_info())

    Topic  Count                                             Name  \
0      -1    987       -1_community_process_participation_project   
1       0    109               0_climate_assembly_change_citizens   
2       1     91           1_budgeting_participatory_community_pb   
3       2     89                    2_community_jury_au_australia   
4       3     64                      3_nbsp_quot_project_process   
5       4     61                   4_vauban_planning_city_airport   
6       5     59                      5_health_care_mental_canada   
7       6     58               6_research_dialogue_public_science   
8       7     56                         7_covid_19_pandemic_fund   
9       8     40                   8_indigenous_de_colombia_peace   
10      9     36                    9_european_eu_europe_citizens   
11     10     35             10_water_watershed_public_management   
12     11     35              11_internet_online_digital_citizens   
13     12     32                  

In [ ]:
# install FAISS (Facebook AI Similarity Search)
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 60.8 MB/s eta 0:00:00


In [ ]:
# store embeddings in FAISS for storing and searching embeddings efficiently

import faiss
import numpy as np

# Convert embeddings to NumPy array
embedding_array = np.array(embeddings).astype("float32")

# Initialize FAISS index
index = faiss.IndexFlatL2(embedding_array.shape[1])  # L2 distance (Euclidean)
index.add(embedding_array)  # Add embeddings to FAISS

# Save FAISS index for later use
faiss.write_index(index, "faiss_index.bin")

In [ ]:
# Encode User Queries & Retrieve Similar Documents

def search_faiss(query, top_k=10):
    # Convert query to embedding
    query_embedding = embedding_model.encode([query]).astype("float32")

    # Search FAISS index
    distances, indices = index.search(query_embedding, top_k)

    # Retrieve matching documents
    results = df.iloc[indices[0]]  # Get rows from original dataframe
    return results

In [ ]:
# Function to apply filters based on LLM-extracted parameters (dynamically)
def apply_filters(results, filters):
    for key, value in filters.items():
        if key in results.columns:
            # Apply case-insensitive filtering on text columns
            results = results[results[key].astype(str).str.contains(str(value), case=False, na=False)]
    return results

In [ ]:
# Sample query
query = "abortion"

# Retrieve top results from FAISS (before filtering)
results = search_faiss(query)

# Print results before filtering to check initial retrieval
print("Results before filtering:", results[["text", "topic"]])

# Identify the most common topics in the retrieved results
top_topics = results["topic"].value_counts()

# Print top topics
print("Top Topics in Results:", top_topics)

# Select the most common topic dynamically from the results (if any)
if not top_topics.empty:
    most_common_topic = top_topics.idxmax()  # Select the most frequent topic
    print(f"Most common topic in results: {most_common_topic}")

    # Apply filter to only show documents with the most common topic
    filters = {"topic": str(most_common_topic)}
    filtered_results = apply_filters(results, filters)

    # Print filtered results
    print("Filtered Results:", filtered_results[["text", "topic"]])
else:
    print("No topics found in the results")

Results before filtering:                                                    text  topic
2107  On June 24 of 2022, the United States Supreme ...     17
1786  Since 2005, women’s movements in Argentina hav...     -1
338   A unique democratic innovation to educate the ...     -1
859   The Thirty-sixth Amendment of the Constitution...     15
921   Human Fertilisation and Embryology Authority (...      6
680   Before legislating on the issue, Public Hearin...     -1
1874                                   Social Activism      -1
1396  From 2001 to 2003, a team of researchers did a...     -1
1892  From 2001 to 2003, a team of researchers did a...     -1
1007  Through dialogue, community scorecards, and ad...     -1
Top Topics in Results: topic
-1     7
 17    1
 15    1
 6     1
Name: count, dtype: int64
Most common topic in results: -1
Filtered Results:                                                    text  topic
1786  Since 2005, women’s movements in Argentina hav...     -1
338   A uniqu